In [1]:
from google.colab import drive
drive.mount('/content/drive')
from fastai.imports import *
from fastai.vision.all import *
from fastai.metrics import error_rate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from fastai.metrics import accuracy_multi

Mounted at /content/drive


In [2]:
ls drive/MyDrive

 Animes.gdoc                                        IMG_20201022_170657_7.jpg
'assignment 2.gdoc'                                 IMG_20201022_170707_8.jpg
'assignment 2.pdf'                                  IMG_20201022_171637_7.jpg
 CoinbaseWalletBackups/                             IMG_20201029_162310_0.jpg
'Colab Notebooks'/                                  IMG_20201029_162509_5.jpg
'confirmation 24 0F 2014.docx'                      IMG_20201104_134014_6.jpg
'Copy of Literature Review Outline Template.gdoc'   IMG_20201104_135315_8.jpg
'Getting started.pdf'                               IMG_20201104_135318_5CS.jpg
 HI.docx                                            IMG_20201104_140414_0.jpg
 IMG_20200905_163122_8.jpg                          IMG_20201104_140417_9.jpg
 IMG_20200905_163756_3.jpg                          IMG_20201104_140423_0.jpg
 IMG_20200905_163931_0.jpg                          IMG_20201104_140619_6.jpg
 IMG_20200905_164002_7.jpg                          IMG_202011

In [3]:
x = '/content/drive/MyDrive/processed_data/train'
path = Path(x)
path.ls

<bound method Path.ls of Path('/content/drive/MyDrive/processed_data/train')>

In [4]:
def custom_show_batch(dl, max_n=6, figsize=(15, 10), n_col=3, row_height=1.0, col_width=1.0, v_spacing=0.5, h_spacing=0.5):
    _, axs = plt.subplots(max_n // n_col, n_col, figsize=(figsize[0] * col_width, figsize[1] * row_height))
    axs = axs.flatten()

    dl.show_batch(max_n=max_n, figsize=(figsize[0] * col_width, figsize[1] * row_height), ctxs=axs)


    # Remove empty subplots
    for empty_ax in axs[max_n:]:
        empty_ax.axis('off')

    # Adjust vertical spacing between subplots
    plt.subplots_adjust(hspace=v_spacing, wspace=h_spacing)

    plt.show()

In [5]:
image_folder = f'{path}/images'
label_folder = f'{path}/labels'

class_values = {
    1: 'Band_Neutrophils',
    2: 'Dyplastic_Cell',
    3: 'Giant_Platelet',
    4: 'Lymphoblast',
    5: 'Lymphocytes',
    6: 'Neutrophil',
    7: 'Promyelocytes',
    8: 'monocytes',
    9: 'myelocytes',
    10: 'not_cell',
    11: 'nucleated_RBC'
}

# Create a DataFrame which will be used in appending dataframes
df = pd.DataFrame(columns = ['image_path', 'class_name', 'x_coordinate', 'y_coordinate', 'width', 'height'])
count = 0
# Loop through the images in the image folder
for image_name in os.listdir(image_folder):
    count += 1
    # Check if the file is an image file
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, image_name)

        # Construct the corresponding label file path
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = os.path.join(label_folder, label_name)

        # Check if the corresponding label file exists
        if os.path.exists(label_path):
            # Read the label information from the text file
            with open(label_path, 'r') as label_file:
                for line in label_file:
                    label_info = line.split()

                    if len(label_info) >= 5:
                        # Extract class index and bounding box coordinates
                        class_index = int(label_info[0])
                        x_coordinate, y_coordinate, width, height = map(float, label_info[1:])

                        # Map class index to class name
                        # class_name = class_values.get(class_index, f'Unknown_{class_index}')

                        # Create a DataFrame for the current label
                        df = pd.concat([
                            df,
                            pd.DataFrame({
                                'image_path': [image_path],
                                'class_name': [class_index],
                                'x_coordinate': [x_coordinate],
                                'y_coordinate': [y_coordinate],
                                'width': [width],
                                'height': [height]
                            })
                        ], ignore_index=True)
                    else:
                        print(f"Invalid label information in file: {label_path}")

# Display the combined data
print(df.head(10))

print(count)

print(df.shape)

                                                                                                      image_path  \
0  /content/drive/MyDrive/processed_data/train/images/6002-17-Dec-20_JPG.rf.3c27f406c782c4e482f581c599eb7d7c.jpg   
1  /content/drive/MyDrive/processed_data/train/images/6003-11-Jan-21_JPG.rf.eb3dcde618330652ee5ffafdebf97aa3.jpg   
2        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg   
3        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg   
4        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg   
5        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg   
6        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg   
7        /content/drive/MyDrive/processed_data/train/images/HA130003_JPG

In [6]:
df['image_path'] = df['image_path'].str.replace(x, '', regex=False)

In [7]:
df.head(10)

,image_path,class_name,x_coordinate,y_coordinate,width,height
0,/images/6002-17-Dec-20_JPG.rf.3c27f406c782c4e482f581c599eb7d7c.jpg,5,0.608796,0.326119,0.042911,0.059707
1,/images/6003-11-Jan-21_JPG.rf.eb3dcde618330652ee5ffafdebf97aa3.jpg,5,0.907792,0.206292,0.058705,0.070688
2,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.417286,0.066698,0.030139,0.038927
3,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.014757,0.451775,0.029514,0.048611
4,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.046574,0.504715,0.028819,0.038881
5,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.084323,0.630309,0.030052,0.041875
6,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.278987,0.394552,0.026238,0.038187
7,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.353773,0.375972,0.027072,0.035123
8,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.505150,0.464120,0.026007,0.039066
9,/images/HA130003_JPG.rf.f3d51ffb24d058ea69b42cfc2e99083d.jpg,3,0.578154,0.588102,0.031591,0.046466


In [8]:
cd /content/drive/MyDrive/processed_data/train

/content/drive/MyDrive/processed_data/train


In [10]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 7.7 MB/s eta 0:00:00


In [11]:
from ultralytics import YOLO

In [12]:
model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:00<00:00, 39.4MB/s]


In [13]:
ls

images/  labels/  yolov8s.pt


In [14]:
model.train(data="../data.yaml", epochs=50)

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=../data.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

100%|██████████| 755k/755k [00:00<00:00, 4.76MB/s]


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 26.8MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅


...

                   all         61        299      0.673      0.374      0.387      0.347



50 epochs completed in 0.526 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.91it/s]


                   all         61        299      0.918      0.376      0.491      0.424
      Band_Neutrophils         61          7          1          0      0.195      0.164
        Dyplastic_Cell         61          7      0.652      0.571      0.549      0.497
           Lymphoblast         61        198      0.962       0.99      0.981      0.878
           Lymphocytes         61          4          1          0     0.0659     0.0521
            Neutrophil         61         54      0.694      0.884      0.848      0.735
         Promyelocytes         61         24      0.957      0.936       0.95      0.807
            myelocytes         61          1          1          0          0          0
              not_cell         61          2          1          0          0          0
         nucleated_RBC         61          2          1          0      0.828      0.679
Speed: 0.3ms preprocess, 4.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  3,  4,  5,  6,  8,  9, 10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ae28b3f0f10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046, 